# 1- import libraries

In [1]:
import numpy as np

# 2- Calculate Mean Squared Error (MSE)

In [2]:
def mse(y):
    return np.mean((y - np.mean(y)) ** 2)

# 3- Calculate Variance Reduction

In [3]:
def variance_reduction(y, y_left, y_right):
    parent_mse = mse(y)
    n = len(y)
    left_mse = mse(y_left)
    right_mse = mse(y_right)
    weighted_mse = (len(y_left) / n) * left_mse + (len(y_right) / n) * right_mse
    return parent_mse - weighted_mse

# 4- Node Class

In [4]:
class DecisionNode:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature  # Index of the feature to split on
        self.threshold = threshold  # Threshold value for splitting
        self.left = left  # Left subtree
        self.right = right  # Right subtree
        self.value = value  # For leaf nodes, this stores the predicted value


# 5- Decision Tree class

In [5]:
class DecisionTreeRegressor:
    def __init__(self, min_samples_split=2, max_depth=100):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.root = None

    # Fit the tree to the data
    def fit(self, X, y):
        self.root = self._build_tree(X, y)

    # Recursively build the tree
    def _build_tree(self, X, y, depth=0):
        num_samples, num_features = X.shape
        
        # Stopping criteria
        if (depth >= self.max_depth or num_samples < self.min_samples_split):
            leaf_value = self._calculate_leaf_value(y)
            return DecisionNode(value=leaf_value)

        # Find the best split
        best_feature, best_threshold = self._best_split(X, y)
        
        # Split the data into left and right subsets
        left_idxs, right_idxs = self._split(X[:, best_feature], best_threshold)
        
        # Recursively build left and right subtrees
        left_subtree = self._build_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right_subtree = self._build_tree(X[right_idxs, :], y[right_idxs], depth + 1)
        
        return DecisionNode(feature=best_feature, threshold=best_threshold, left=left_subtree, right=right_subtree)

    # Find the best feature and threshold to split the data
    def _best_split(self, X, y):
        best_gain = -float('inf')
        best_feature, best_threshold = None, None

        for feature in range(X.shape[1]):
            X_column = X[:, feature]
            thresholds = np.unique(X_column)
            
            for threshold in thresholds:
                left_idxs, right_idxs = self._split(X_column, threshold)
                if len(left_idxs) == 0 or len(right_idxs) == 0:
                    continue

                y_left, y_right = y[left_idxs], y[right_idxs]
                gain = variance_reduction(y, y_left, y_right)

                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold

        return best_feature, best_threshold

    # Split the dataset into two groups
    def _split(self, X_column, threshold):
        left_idxs = np.argwhere(X_column <= threshold).flatten()
        right_idxs = np.argwhere(X_column > threshold).flatten()
        return left_idxs, right_idxs

    # Calculate the leaf node value (mean of the target values in the node)
    def _calculate_leaf_value(self, y):
        return np.mean(y)

    # Traverse the tree to make a prediction
    def _traverse_tree(self, x, node):
        if node.value is not None:
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        else:
            return self._traverse_tree(x, node.right)

    # Predict the target values for a set of samples
    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])


# 6- Example

In [6]:
if __name__ == "__main__":
    # Example dataset (house sizes and corresponding prices)
    X = np.array([[1400], [1600], [1700], [1875], [1100], [1550], [2350], [2450], [1425], [1700]])
    y = np.array([245000, 312000, 279000, 308000, 199000, 219000, 405000, 324000, 319000, 255000])

    # Initialize and fit the decision tree regressor
    tree = DecisionTreeRegressor(max_depth=3)
    tree.fit(X, y)

    # Make predictions
    X_test = np.array([[1500], [1800], [2200]])  # Test data (new house sizes)
    predictions = tree.predict(X_test)
    print("Predictions:", predictions)

Predictions: [274600. 274600. 405000.]
